<a href="https://colab.research.google.com/github/Nicolas-Ayala-Collazos/ML/blob/main/AguasAbajo_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx
import geopandas as gpd

def encontrar_nodos_aguas_abajo(G, nodo_fuente):
    """
    Encuentra todos los nodos aguas abajo de un nodo dado en el grafo.

    Args:
    G (nx.Graph): El grafo.
    nodo_fuente (str): El nodo desde el cual comenzar la búsqueda.

    Returns:
    dict: Un diccionario de nodos aguas abajo y sus caminos desde el nodo fuente.
    """
    nodos_aguas_abajo = {}
    for nodo in G.nodes:
        if nx.has_path(G, nodo_fuente, nodo) and nodo != nodo_fuente:
            nodos_aguas_abajo[nodo] = nx.shortest_path(G, nodo_fuente, nodo)
    return nodos_aguas_abajo

# Cargar el shapefile
shapefile_path = '/content/GALAN.shp'
shapefile = gpd.read_file(shapefile_path)

# Crear el grafo
G = nx.Graph()
for segment in shapefile['LINE_SEGME']:
    nodes = segment.split('-')[:2]
    nodes = [node[:-2] if node.endswith('-1') else node for node in nodes]
    G.add_edge(nodes[0], nodes[1])

# Seleccionar un nodo y encontrar nodos aguas abajo
nodo_seleccionado = 'CD10.3.019031'
nodos_aguas_abajo = encontrar_nodos_aguas_abajo(G, nodo_seleccionado)

# Imprimir los resultados
for nodo, camino in nodos_aguas_abajo.items():
    print(f"Nodo: {nodo}, Camino: {camino}")

# Asignar colores a los nodos según sus prefijos
node_colors = []
for node in G.nodes():
    if node.startswith('CD00.1.'):
        node_colors.append('orange')  # Naranja café para nodos que comienzan con 'CD00.1.'
    elif node.startswith('CD10.3.'):
        node_colors.append('green')  # Verde para nodos que comienzan con 'CD10.3.'
    elif node.startswith('CD10.2.'):
        node_colors.append('red')  # Rojo para nodos que comienzan con 'CD10.2.'
    elif node.startswith('CD10.4.'):
        node_colors.append('blue')  # Azul para nodos que comienzan con 'CD10.4.'
    else:
        node_colors.append('gray')  # Gris para cualquier otro nodo

# Seleccionar un nodo y cambiar su color a fosforescente
nodo_seleccionado = 'CD10.3.019031'
node_colors_highlighted = ['lime' if node == nodo_seleccionado else color for node, color in zip(G.nodes(), node_colors)]

# Utilizar el layout de Kamada-Kawai para el grafo
pos_kamada_kawai = nx.kamada_kawai_layout(G)

# Configurar y dibujar el grafo
plt.figure(figsize=(15, 15))
nx.draw(G, pos_kamada_kawai, with_labels=True, node_color=node_colors_highlighted, edge_color="gray", font_size=8, node_size=50)
plt.title("Grafo con Layout de Kamada-Kawai y Nodo Destacado en Color Fosforescente")
plt.show()



ERROR:fiona._env:/content/GALAN.shp: No such file or directory


DriverError: /content/GALAN.shp: No such file or directory

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx

# Definir la función para explorar conexiones al camino
def explorar_conexiones_al_camino(camino, grafo):
    nodos_conectados = set()
    aristas_conectadas = set()

    for nodo in camino:
        # Agregar nodos y aristas adyacentes al nodo actual
        for vecino in grafo.neighbors(nodo):
            nodos_conectados.add(vecino)
            aristas_conectadas.add((nodo, vecino))

    return nodos_conectados, aristas_conectadas

# Definir la función de barrido eléctrico extendido
def barrido_electrico_extendido(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio in grafo and nodo_fin in grafo:
        if nx.has_path(grafo, nodo_inicio, nodo_fin):
            camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
            nodos_conectados, aristas_conectadas = explorar_conexiones_al_camino(camino, grafo)
            return camino, nodos_conectados, aristas_conectadas
        else:
            return "No hay un camino entre los nodos.", set(), set()
    else:
        return "Uno o ambos nodos no existen en el grafo.", set(), set()

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
lista_nodos = list(G.nodes)
nodo_inicio_input = widgets.Combobox(options=lista_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
nodo_fin_input = widgets.Combobox(options=lista_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Botón para ejecutar el barrido eléctrico
boton_barrido = widgets.Button(description='Realizar Barrido')

# Salida para mostrar los resultados
salida_barrido = widgets.Output()

def on_boton_barrido_clicked(b):
    # Limpiar la salida anterior
    salida_barrido.clear_output()

    # Obtener los nodos de los widgets
    nodo_inicio = nodo_inicio_input.value
    nodo_fin = nodo_fin_input.value

    # Realizar el barrido eléctrico extendido
    camino, nodos_conectados, aristas_conectadas = barrido_electrico_extendido(nodo_inicio, nodo_fin, G)

    # Mostrar los resultados y el grafo
    with salida_barrido:
        print("Camino:", camino)
        print("Nodos Conectados:", nodos_conectados)
        print("Aristas Conectadas:", aristas_conectadas)

        # Asignar colores a los nodos según sus prefijos y destacar el nodo de inicio y fin
        node_colors = []
        for node in G.nodes():
            if node == nodo_inicio:
                node_colors.append('lime')  # Fosforescente para el nodo de inicio
            elif node == nodo_fin:
                node_colors.append('black')  # Negro para el nodo de fin
            elif node.startswith('CD00.1.'):
                node_colors.append('orange')  # Naranja café
            elif node.startswith('CD10.3.'):
                node_colors.append('green')  # Verde
            elif node.startswith('CD10.2.'):
                node_colors.append('red')  # Rojo
            elif node.startswith('CD10.4.'):
                node_colors.append('blue')  # Azul
            else:
                node_colors.append('gray')  # Gris

        # Dibujar el grafo
        plt.figure(figsize=(12, 12))
        nx.draw(G, pos_kamada_kawai, with_labels=True, node_color=node_colors, edge_color="gray", font_size=8, node_size=50)
        plt.show()

# Asignar la acción al botón
boton_barrido.on_click(on_boton_barrido_clicked)

# Mostrar los widgets
display(nodo_inicio_input, nodo_fin_input, boton_barrido, salida_barrido)


Combobox(value='', description='Nodo Inicio:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', …

Combobox(value='', description='Nodo Fin:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', 'CD…

Button(description='Realizar Barrido', style=ButtonStyle())

Output()

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx
import geopandas as gpd

# Cargar el shapefile
shapefile_path = '/content/GALAN.shp'  # Reemplaza con la ruta correcta a tu archivo shapefile
shapefile = gpd.read_file(shapefile_path)

# Crear el grafo a partir del shapefile
G = nx.Graph()
for segment in shapefile['LINE_SEGME']:
    nodes = segment.split('-')[:2]
    nodes = [node[:-2] if node.endswith('-1') else node for node in nodes]
    G.add_edge(nodes[0], nodes[1])

# Aquí generamos el layout de Kamada-Kawai
pos_kamada_kawai = nx.kamada_kawai_layout(G)
# Definir la función para explorar conexiones al camino
def explorar_ramales_del_camino(camino, grafo):
    nodos_ramales = set()
    aristas_ramales = set()

    for nodo_camino in camino:
        for vecino in grafo.neighbors(nodo_camino):
            if vecino not in camino:
                nodos_ramales.add(vecino)
                aristas_ramales.add((nodo_camino, vecino))

    return nodos_ramales, aristas_ramales

    for nodo_camino in camino:
        explorar_desde_nodo(nodo_camino)

    return nodos_ramales, aristas_ramales
# Definir la función de barrido eléctrico extendido
def barrido_electrico_extendido(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio in grafo and nodo_fin in grafo:
        if nx.has_path(grafo, nodo_inicio, nodo_fin):
            camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
            nodos_conectados, aristas_conectadas = explorar_ramales_del_camino(camino, grafo)
            return camino, nodos_conectados, aristas_conectadas
        else:
            return "No hay un camino entre los nodos.", set(), set()
    else:
        return "Uno o ambos nodos no existen en el grafo.", set(), set()

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
lista_nodos = list(G.nodes)
nodo_inicio_input = widgets.Combobox(options=lista_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
nodo_fin_input = widgets.Combobox(options=lista_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Botón para ejecutar el barrido eléctrico
boton_barrido = widgets.Button(description='Realizar Barrido')

# Salida para mostrar los resultados
salida_barrido = widgets.Output()

def oscurecer_color(color):
    """Devuelve una versión más oscura del color dado."""
    colores = {
        'green': 'darkgreen',
        'red': 'darkred',
        'blue': 'darkblue',
        'orange': 'darkorange',
        'gray': 'dimgray',
        'lime': 'limegreen',
        'black': 'black'
    }
    return colores.get(color, color)

def on_boton_barrido_clicked(b):
    # Limpiar la salida anterior
    salida_barrido.clear_output()

    # Obtener los nodos de los widgets
    nodo_inicio = nodo_inicio_input.value
    nodo_fin = nodo_fin_input.value

    # Realizar el barrido eléctrico extendido
    camino, nodos_conectados, aristas_conectadas = barrido_electrico_extendido(nodo_inicio, nodo_fin, G)

    # Mostrar los resultados y el grafo
    with salida_barrido:
        print("Camino:", camino)
        print("Nodos Conectados:", nodos_conectados)
        print("Aristas Conectadas:", aristas_conectadas)

        # Asignar colores a los nodos según sus prefijos y destacar el nodo de inicio y fin
        node_colors = []
        for node in G.nodes():
            color_base = 'gray'  # Color por defecto
            if node.startswith('CD00.1.'):
                color_base = 'orange'
            elif node.startswith('CD10.3.'):
                color_base = 'green'
            elif node.startswith('CD10.2.'):
                color_base = 'red'
            elif node.startswith('CD10.4.'):
                color_base = 'blue'

            # Oscurecer los nodos conectados al camino
            if node in nodos_conectados:
                color = oscurecer_color(color_base)
            else:
                color = color_base

            # Colores especiales para nodo de inicio y fin
            if node == nodo_inicio:
                color = 'lime'
            elif node == nodo_fin:
                color = 'black'

            node_colors.append(color)

        # Dibujar el grafo
        plt.figure(figsize=(12, 12))
        nx.draw(G, pos_kamada_kawai, with_labels=True, node_color=node_colors, edge_color="gray", font_size=8, node_size=50)
        plt.show()

# Asignar la acción al botón
boton_barrido.on_click(on_boton_barrido_clicked)

# Mostrar los widgets
display(nodo_inicio_input, nodo_fin_input, boton_barrido, salida_barrido)


Combobox(value='', description='Nodo Inicio:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', …

Combobox(value='', description='Nodo Fin:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', 'CD…

Button(description='Realizar Barrido', style=ButtonStyle())

Output()

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx
import geopandas as gpd

# Cargar el shapefile
shapefile_path = '/content/GALAN.shp'  # Reemplaza con la ruta correcta a tu archivo shapefile
shapefile = gpd.read_file(shapefile_path)

# Crear el grafo a partir del shapefile
G = nx.Graph()
for segment in shapefile['LINE_SEGME']:
    nodes = segment.split('-')[:2]
    nodes = [node[:-2] if node.endswith('-1') else node for node in nodes]
    G.add_edge(nodes[0], nodes[1])

# Aquí generamos el layout de Kamada-Kawai
pos_kamada_kawai = nx.kamada_kawai_layout(G)
def explorar_ramales_del_camino(nodo_inicio, nodo_fin, grafo):
    nodos_explorados = set()
    aristas_exploradas = set()

    # Exploración en profundidad desde cada nodo en el camino
    def explorar_desde_nodo(nodo_actual, visitados):
        if nodo_actual == nodo_fin:
            return
        for vecino in grafo.neighbors(nodo_actual):
            if vecino not in visitados and vecino != nodo_fin:
                nodos_explorados.add(vecino)
                aristas_exploradas.add((nodo_actual, vecino))
                visitados.add(vecino)
                explorar_desde_nodo(vecino, visitados)

    # Iniciar la exploración desde el nodo inicial
    explorar_desde_nodo(nodo_inicio, set([nodo_inicio]))

    return nodos_explorados, aristas_exploradas
# Definir la función de barrido eléctrico extendido

def obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return set(), set()

    nodos_en_camino = set(camino)
    aristas_en_camino = set()

    for i in range(len(camino) - 1):
        aristas_en_camino.add((camino[i], camino[i + 1]))

    # Agregar nodos adyacentes al camino
    for nodo in camino:
        for vecino in grafo.neighbors(nodo):
            if vecino not in nodos_en_camino:
                nodos_en_camino.add(vecino)
                aristas_en_camino.add((nodo, vecino))

    return nodos_en_camino, aristas_en_camino

# Función de barrido eléctrico extendido
def barrido_electrico_extendido(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return "Uno o ambos nodos no están en el grafo", set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return "No hay un camino entre los nodos.", set(), set()

    nodos_conectados, aristas_conectadas = obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo)
    return camino, nodos_conectados, aristas_conectadas

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
lista_nodos = list(G.nodes)
nodo_inicio_input = widgets.Combobox(options=lista_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
nodo_fin_input = widgets.Combobox(options=lista_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Botón para ejecutar el barrido eléctrico
boton_barrido = widgets.Button(description='Realizar Barrido')

# Salida para mostrar los resultados
salida_barrido = widgets.Output()

def oscurecer_color(color):
    """Devuelve una versión más oscura del color dado."""
    colores = {
        'green': 'darkgreen',
        'red': 'darkred',
        'blue': 'darkblue',
        'orange': 'darkorange',
        'gray': 'dimgray',
        'lime': 'limegreen',
        'black': 'black'
    }
    return colores.get(color, color)

def on_boton_barrido_clicked(b):
    # Limpiar la salida anterior
    salida_barrido.clear_output()

    # Obtener los nodos de los widgets
    nodo_inicio = nodo_inicio_input.value
    nodo_fin = nodo_fin_input.value

    # Realizar el barrido eléctrico extendido
    camino, nodos_conectados, aristas_conectadas = barrido_electrico_extendido(nodo_inicio, nodo_fin, G)

    # Mostrar los resultados y el grafo
    with salida_barrido:
        print("Camino:", camino)
        print("Nodos Conectados:", nodos_conectados)
        print("Aristas Conectadas:", aristas_conectadas)

        # Asignar colores a los nodos según sus prefijos y destacar el nodo de inicio y fin
        node_colors = []
        for node in G.nodes():
            color_base = 'gray'  # Color por defecto
            if node.startswith('CD00.1.'):
                color_base = 'orange'
            elif node.startswith('CD10.3.'):
                color_base = 'green'
            elif node.startswith('CD10.2.'):
                color_base = 'red'
            elif node.startswith('CD10.4.'):
                color_base = 'blue'

            # Oscurecer los nodos conectados al camino
            if node in nodos_conectados:
                color = oscurecer_color(color_base)
            else:
                color = color_base

            # Colores especiales para nodo de inicio y fin
            if node == nodo_inicio:
                color = 'lime'
            elif node == nodo_fin:
                color = 'black'

            node_colors.append(color)

        # Dibujar el grafo
        plt.figure(figsize=(12, 12))
        nx.draw(G, pos_kamada_kawai, with_labels=True, node_color=node_colors, edge_color="gray", font_size=8, node_size=50)
        plt.show()

# Asignar la acción al botón
boton_barrido.on_click(on_boton_barrido_clicked)

# Mostrar los widgets
display(nodo_inicio_input, nodo_fin_input, boton_barrido, salida_barrido)


Combobox(value='', description='Nodo Inicio:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', …

Combobox(value='', description='Nodo Fin:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', 'CD…

Button(description='Realizar Barrido', style=ButtonStyle())

Output()

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx
import geopandas as gpd
import folium
# Cargar el shapefile
shapefile_path = '/content/GALAN.shp'  # Reemplaza con la ruta correcta a tu archivo shapefile
shapefile = gpd.read_file(shapefile_path)

# Crear el grafo a partir del shapefile
G = nx.Graph()
for segment in shapefile['LINE_SEGME']:
    nodes = segment.split('-')[:2]
    nodes = [node[:-2] if node.endswith('-1') else node for node in nodes]
    G.add_edge(nodes[0], nodes[1])

# Aquí generamos el layout de Kamada-Kawai
pos_kamada_kawai = nx.kamada_kawai_layout(G)
def explorar_ramales_del_camino(nodo_inicio, nodo_fin, grafo):
    nodos_explorados = set()
    aristas_exploradas = set()

    # Exploración en profundidad desde cada nodo en el camino
    def explorar_desde_nodo(nodo_actual, visitados):
        if nodo_actual == nodo_fin:
            return
        for vecino in grafo.neighbors(nodo_actual):
            if vecino not in visitados and vecino != nodo_fin:
                nodos_explorados.add(vecino)
                aristas_exploradas.add((nodo_actual, vecino))
                visitados.add(vecino)
                explorar_desde_nodo(vecino, visitados)

    # Iniciar la exploración desde el nodo inicial
    explorar_desde_nodo(nodo_inicio, set([nodo_inicio]))

    return nodos_explorados, aristas_exploradas
# Definir la función de barrido eléctrico extendido

def obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return set(), set()

    nodos_en_camino = set(camino)
    aristas_en_camino = set()

    for i in range(len(camino) - 1):
        aristas_en_camino.add((camino[i], camino[i + 1]))

    # Agregar nodos adyacentes al camino
    for nodo in camino:
        for vecino in grafo.neighbors(nodo):
            if vecino not in nodos_en_camino:
                nodos_en_camino.add(vecino)
                aristas_en_camino.add((nodo, vecino))

    return nodos_en_camino, aristas_en_camino

# Función de barrido eléctrico extendido
def barrido_electrico_extendido(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return "Uno o ambos nodos no están en el grafo", set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return "No hay un camino entre los nodos.", set(), set()

    nodos_conectados, aristas_conectadas = obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo)
    return camino, nodos_conectados, aristas_conectadas

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
lista_nodos = list(G.nodes)
nodo_inicio_input = widgets.Combobox(options=lista_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
nodo_fin_input = widgets.Combobox(options=lista_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Botón para ejecutar el barrido eléctrico
boton_barrido = widgets.Button(description='Realizar Barrido')

# Salida para mostrar los resultados
salida_barrido = widgets.Output()

def oscurecer_color(color):
    """Devuelve una versión más oscura del color dado."""
    colores = {
        'green': 'darkgreen',
        'red': 'darkred',
        'blue': 'darkblue',
        'orange': 'darkorange',
        'gray': 'dimgray',
        'lime': 'limegreen',
        'black': 'black'
    }
    return colores.get(color, color)

caminos_acumulados = []
nodos_conectados_acumulados = set()

def on_boton_barrido_clicked(b):
    # Limpiar la salida anterior
    salida_barrido.clear_output()

    # Obtener los nodos de los widgets
    nodo_inicio = nodo_inicio_input.value
    nodo_fin = nodo_fin_input.value

    # Realizar el barrido eléctrico extendido
    camino, nodos_conectados, aristas_conectadas = barrido_electrico_extendido(nodo_inicio, nodo_fin, G)
       # Acumular los resultados
    caminos_acumulados.append(camino)
    nodos_conectados_acumulados.update(nodos_conectados)

    # Mostrar los resultados y el grafo
    with salida_barrido:
        print("Camino Actual:", camino)
        print("Todos los Caminos:", caminos_acumulados)
        print("Nodos Conectados Acumulados:", nodos_conectados_acumulados)

        # Asignar colores a los nodos según sus prefijos y destacar el nodo de inicio y fin
        node_colors = []
        for node in G.nodes():
            color_base = 'gray'  # Color por defecto
            if node.startswith('CD00.1.'):
                color_base = 'orange'
            elif node.startswith('CD10.3.'):
                color_base = 'green'
            elif node.startswith('CD10.2.'):
                color_base = 'red'
            elif node.startswith('CD10.4.'):
                color_base = 'blue'

            # Oscurecer los nodos conectados al camino
            if node in nodos_conectados_acumulados:
                color = oscurecer_color(color_base)
            else:
                color = color_base

            # Colores especiales para nodo de inicio y fin
            if node == nodo_inicio:
                color = 'lime'
            elif node == nodo_fin:
                color = 'black'

            node_colors.append(color)

        # Dibujar el grafo
        plt.figure(figsize=(12, 12))
        nx.draw(G, pos_kamada_kawai, with_labels=True, node_color=node_colors, edge_color="gray", font_size=8, node_size=50)
        plt.show()

# Asignar la acción al botón
boton_barrido.on_click(on_boton_barrido_clicked)

# Mostrar los widgets
display(nodo_inicio_input, nodo_fin_input, boton_barrido, salida_barrido)





Combobox(value='', description='Nodo Inicio:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', …

Combobox(value='', description='Nodo Fin:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', 'CD…

Button(description='Realizar Barrido', style=ButtonStyle())

Output()

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx
import geopandas as gpd
import folium
# Cargar el shapefile
shapefile_path = '/content/GALAN.shp'  # Reemplaza con la ruta correcta a tu archivo shapefile
shapefile = gpd.read_file(shapefile_path)

# Crear el grafo a partir del shapefile
G = nx.Graph()
for segment in shapefile['LINE_SEGME']:
    nodes = segment.split('-')[:2]
    nodes = [node[:-2] if node.endswith('-1') else node for node in nodes]
    G.add_edge(nodes[0], nodes[1])

# Aquí generamos el layout de Kamada-Kawai
pos_kamada_kawai = nx.kamada_kawai_layout(G)
def explorar_ramales_del_camino(nodo_inicio, nodo_fin, grafo):
    nodos_explorados = set()
    aristas_exploradas = set()

    # Exploración en profundidad desde cada nodo en el camino
    def explorar_desde_nodo(nodo_actual, visitados):
        if nodo_actual == nodo_fin:
            return
        for vecino in grafo.neighbors(nodo_actual):
            if vecino not in visitados and vecino != nodo_fin:
                nodos_explorados.add(vecino)
                aristas_exploradas.add((nodo_actual, vecino))
                visitados.add(vecino)
                explorar_desde_nodo(vecino, visitados)

    # Iniciar la exploración desde el nodo inicial
    explorar_desde_nodo(nodo_inicio, set([nodo_inicio]))

    return nodos_explorados, aristas_exploradas
# Definir la función de barrido eléctrico extendido

def obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return set(), set()

    nodos_en_camino = set(camino)
    aristas_en_camino = set()

    for i in range(len(camino) - 1):
        aristas_en_camino.add((camino[i], camino[i + 1]))

    # Agregar nodos adyacentes al camino
    for nodo in camino:
        for vecino in grafo.neighbors(nodo):
            if vecino not in nodos_en_camino:
                nodos_en_camino.add(vecino)
                aristas_en_camino.add((nodo, vecino))

    return nodos_en_camino, aristas_en_camino

# Función de barrido eléctrico extendido
def barrido_electrico_extendido(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return "Uno o ambos nodos no están en el grafo", set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return "No hay un camino entre los nodos.", set(), set()

    nodos_conectados, aristas_conectadas = obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo)
    return camino, nodos_conectados, aristas_conectadas

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
lista_nodos = list(G.nodes)
nodo_inicio_input = widgets.Combobox(options=lista_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
nodo_fin_input = widgets.Combobox(options=lista_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Botón para ejecutar el barrido eléctrico
boton_barrido = widgets.Button(description='Realizar Barrido')

# Salida para mostrar los resultados
salida_barrido = widgets.Output()

def oscurecer_color(color):
    """Devuelve una versión más oscura del color dado."""
    colores = {
        'green': 'darkgreen',
        'red': 'darkred',
        'blue': 'darkblue',
        'orange': 'darkorange',
        'gray': 'dimgray',
        'lime': 'limegreen',
        'black': 'black'
    }
    return colores.get(color, color)

caminos_acumulados = []
nodos_conectados_acumulados = set()

def on_boton_barrido_clicked(b):
    # Limpiar la salida anterior
    salida_barrido.clear_output()

    # Obtener los nodos de los widgets
    nodo_inicio = nodo_inicio_input.value
    nodo_fin = nodo_fin_input.value

    # Realizar el barrido eléctrico extendido
    camino, nodos_conectados, aristas_conectadas = barrido_electrico_extendido(nodo_inicio, nodo_fin, G)
       # Acumular los resultados
    caminos_acumulados.append(camino)
    nodos_conectados_acumulados.update(nodos_conectados)

    # Mostrar los resultados y el grafo
    with salida_barrido:
        print("Camino Actual:", camino)
        print("Todos los Caminos:", caminos_acumulados)
        print("Nodos Conectados Acumulados:", nodos_conectados_acumulados)

        # Asignar colores a los nodos según sus prefijos y destacar el nodo de inicio y fin
        node_colors = []
        for node in G.nodes():
            color_base = 'gray'  # Color por defecto
            if node.startswith('CD00.1.'):
                color_base = 'orange'
            elif node.startswith('CD10.3.'):
                color_base = 'green'
            elif node.startswith('CD10.2.'):
                color_base = 'red'
            elif node.startswith('CD10.4.'):
                color_base = 'blue'

            # Oscurecer los nodos conectados al camino
            if node in nodos_conectados_acumulados:
                color = oscurecer_color(color_base)
            else:
                color = color_base

            # Colores especiales para nodo de inicio y fin
            if node == nodo_inicio:
                color = 'lime'
            elif node == nodo_fin:
                color = 'black'

            node_colors.append(color)

        # Dibujar el grafo
        plt.figure(figsize=(12, 12))
        nx.draw(G, pos_kamada_kawai, with_labels=True, node_color=node_colors, edge_color="gray", font_size=8, node_size=50)
        plt.show()

# Asignar la acción al botón
boton_barrido.on_click(on_boton_barrido_clicked)

# Mostrar los widgets
display(nodo_inicio_input, nodo_fin_input, boton_barrido, salida_barrido)





Combobox(value='', description='Nodo Inicio:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', …

Combobox(value='', description='Nodo Fin:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', 'CD…

Button(description='Realizar Barrido', style=ButtonStyle())

Output()

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx
import geopandas as gpd
import folium
# Cargar el shapefile
shapefile_path = '/content/GALAN.shp'  # Reemplaza con la ruta correcta a tu archivo shapefile
shapefile = gpd.read_file(shapefile_path)

# Crear el grafo a partir del shapefile
G = nx.Graph()
for segment in shapefile['LINE_SEGME']:
    nodes = segment.split('-')[:2]
    nodes = [node[:-2] if node.endswith('-1') else node for node in nodes]
    G.add_edge(nodes[0], nodes[1])

# Aquí generamos el layout de Kamada-Kawai
pos_kamada_kawai = nx.kamada_kawai_layout(G)
def explorar_ramales_del_camino(nodo_inicio, nodo_fin, grafo):
    nodos_explorados = set()
    aristas_exploradas = set()

    # Exploración en profundidad desde cada nodo en el camino
    def explorar_desde_nodo(nodo_actual, visitados):
        if nodo_actual == nodo_fin:
            return
        for vecino in grafo.neighbors(nodo_actual):
            if vecino not in visitados and vecino != nodo_fin:
                nodos_explorados.add(vecino)
                aristas_exploradas.add((nodo_actual, vecino))
                visitados.add(vecino)
                explorar_desde_nodo(vecino, visitados)

    # Iniciar la exploración desde el nodo inicial
    explorar_desde_nodo(nodo_inicio, set([nodo_inicio]))

    return nodos_explorados, aristas_exploradas
# Definir la función de barrido eléctrico extendido

def obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return set(), set()

    nodos_en_camino = set(camino)
    aristas_en_camino = set()

    for i in range(len(camino) - 1):
        aristas_en_camino.add((camino[i], camino[i + 1]))

    # Agregar nodos adyacentes al camino
    for nodo in camino:
        for vecino in grafo.neighbors(nodo):
            if vecino not in nodos_en_camino:
                nodos_en_camino.add(vecino)
                aristas_en_camino.add((nodo, vecino))

    return nodos_en_camino, aristas_en_camino

# Función de barrido eléctrico extendido
def barrido_electrico_extendido(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return "Uno o ambos nodos no están en el grafo", set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return "No hay un camino entre los nodos.", set(), set()

    nodos_conectados, aristas_conectadas = obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo)
    return camino, nodos_conectados, aristas_conectadas

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
lista_nodos = list(G.nodes)
nodo_inicio_input = widgets.Combobox(options=lista_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
nodo_fin_input = widgets.Combobox(options=lista_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Botón para ejecutar el barrido eléctrico
boton_barrido = widgets.Button(description='Realizar Barrido')

# Salida para mostrar los resultados
salida_barrido = widgets.Output()

def oscurecer_color(color):
    """Devuelve una versión más oscura del color dado."""
    colores = {
        'green': 'darkgreen',
        'red': 'darkred',
        'blue': 'darkblue',
        'orange': 'darkorange',
        'gray': 'dimgray',
        'lime': 'limegreen',
        'black': 'black'
    }
    return colores.get(color, color)

caminos_acumulados = []
nodos_conectados_acumulados = set()

def on_boton_barrido_clicked(b):
    # Limpiar la salida anterior
    salida_barrido.clear_output()

    # Obtener los nodos de los widgets
    nodo_inicio = nodo_inicio_input.value
    nodo_fin = nodo_fin_input.value

    # Realizar el barrido eléctrico extendido
    camino, nodos_conectados, aristas_conectadas = barrido_electrico_extendido(nodo_inicio, nodo_fin, G)
       # Acumular los resultados
    caminos_acumulados.append(camino)
    nodos_conectados_acumulados.update(nodos_conectados)

    # Mostrar los resultados y el grafo
    with salida_barrido:
        print("Camino Actual:", camino)
        print("Todos los Caminos:", caminos_acumulados)
        print("Nodos Conectados Acumulados:", nodos_conectados_acumulados)

        # Asignar colores a los nodos según sus prefijos y destacar el nodo de inicio y fin
        node_colors = []
        for node in G.nodes():
            color_base = 'gray'  # Color por defecto
            if node.startswith('CD00.1.'):
                color_base = 'orange'
            elif node.startswith('CD10.3.'):
                color_base = 'green'
            elif node.startswith('CD10.2.'):
                color_base = 'red'
            elif node.startswith('CD10.4.'):
                color_base = 'blue'

            # Oscurecer los nodos conectados al camino
            if node in nodos_conectados_acumulados:
                color = oscurecer_color(color_base)
            else:
                color = color_base

            # Colores especiales para nodo de inicio y fin
            if node == nodo_inicio:
                color = 'lime'
            elif node == nodo_fin:
                color = 'black'

            node_colors.append(color)

        # Dibujar el grafo
        plt.figure(figsize=(12, 12))
        nx.draw(G, pos_kamada_kawai, with_labels=True, node_color=node_colors, edge_color="gray", font_size=8, node_size=50)
        plt.show()

# Asignar la acción al botón
boton_barrido.on_click(on_boton_barrido_clicked)

# Mostrar los widgets
display(nodo_inicio_input, nodo_fin_input, boton_barrido, salida_barrido)





Combobox(value='', description='Nodo Inicio:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', …

Combobox(value='', description='Nodo Fin:', ensure_option=True, options=('CD10.3.019035', 'CD10.3.019032', 'CD…

Button(description='Realizar Barrido', style=ButtonStyle())

Output()